In [110]:
from transformers import MarianMTModel, MarianTokenizer, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings


In [111]:
model_id = "Helsinki-NLP/opus-mt-en-gmq"

model = MarianMTModel.from_pretrained(model_id, output_attentions=True)
tokenizer = MarianTokenizer.from_pretrained(model_id)


In [134]:
txt = "would you mind if I gave it another shot?"
encoder_input_ids = tokenizer(f">>nob<< {txt}", return_tensors="pt", add_special_tokens=True).input_ids
token_ids = model.generate(encoder_input_ids, length_penalty=-1.0, max_length=len(txt))
translated = tokenizer.decode(token_ids[0], skip_special_tokens=True)
print(translated)
with tokenizer.as_target_tokenizer():
    decoder_input_ids = tokenizer(translated, return_tensors="pt", add_special_tokens=True).input_ids
outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)
encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0][:])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0][:])
# rework the decoder attentions to skip the first value:
dec_att = [att[:, :, :-1, :-1] for att in outputs.decoder_attentions]
enc_att = [att[:, :, 1:-1, 1:-1] for att in outputs.encoder_attentions]

# how can the cross attention be visualized, when we have removed tokens:
enc_txt = encoder_text[1:-1]
dec_txt = decoder_text[:-1]
print(f"Enc: {len(enc_txt)}, Dec: {len(dec_txt)}")
cross_att = [att[:, :, :-1, 1:-1] for att in outputs.cross_attentions]
print(cross_att[0].shape)

model_view(
    # encoder_attention=enc_att,
    # decoder_attention=dec_att,
    cross_attention=cross_att,
    encoder_tokens=enc_txt,
    decoder_tokens=dec_txt,
    display_mode="light",
    include_layers=[0],
    include_heads=[4],
)

Er det i orden hvis jeg gir den et nytt forsøk?
Enc: 10, Dec: 12
torch.Size([1, 8, 12, 10])


<IPython.core.display.Javascript object>

# Some findings:

Head #4 was found to convey a good explanation of the attention towards concepts and semantic meaning. We thus illustrate the different compression models on the same sentence, to illustrate how the models attend differently to parts of the source sentence.